<a href="https://colab.research.google.com/github/cerr/pycerr-notebooks/blob/main/autosegment_CT_HeadAndNeck_OARs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction

In this tutorial, we will demonstrate how to apply pre-trained AI models to segment OARs for RT treatment planning on head and neck CT scans.

## Requirements
* Python>=3.8
* Although these models can be run on a CPU, GPU access is recommended for efficiency.
  To use a GPU on Colab:  Select `Runtime` > `Change runtime type > Select GPU `
* Data I/O, pre- and post-processing are performed using [***pyCERR***](https://github.com/cerr/pyCERR) .

## I/O
* **Input**: DICOM-format head and neck CT scans(s).  
  
* **Output**: DICOM RTStruct-format segmentations.  
  
  
  Input data should be organized as: one directory of DICOM images per patient.      
  
    
    Input dir
            |------Pat1  
                      |------img1.dcm  
                             img2.dcm  
                             ....  
                             ....  
            |-----Pat2  
                     |------img1.dcm  
                            img2.dcm  
                            ....  
                            ....  


## AI models
* The segmentation models used here were trained and validated on CT scans used for RT planning. Its performance on diagnostic scans is expected to be sub-optimal.  
* Trained models are packaged as a Conda environment archive containing  python libraries and other dependencies.

### Installing models and dependencies

* Installation is performed using CERR's [***model installer***]( https://github.com/cerr/model_installer).  

* A Conda archive containing dependencies is downloaded to the `conda-pack`   
  sub-directory of a configurable `scriptInstallDir`.  
  By default `condaEnvPath = '/content/CT_HeadAndNeck_OARs/conda-pack'`
  
*  Inference scripts are located at
  `deeplabScriptPath = os.path.join(condaEnvPath,'model_wrapper', run_inference_deeplab_nii.py.py')`   
  `selfAttnScriptPath = os.path.join(condaEnvPath,'model_wrapper', run_inference_selfattn_nii.py.py')`  

### Running the models
```python
!python {deeplabScriptPath} {input_directory} {temp_session_directory} {output_directory}
!python {selfAttnScriptPath} {input_directory} {temp_session_directory} {output_directory}
```

## License

By downloading the software you are agreeing to the following terms and conditions as well as to the Terms of Use of CERR software.

**`THE SOFTWARE IS PROVIDED "AS IS" AND CERR DEVELOPMENT TEAM AND ITS COLLABORATORS DO NOT MAKE ANY WARRANTY, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO WARRANTIES OF MERCHANTABILITY AND FITNESS FOR A PARTICULAR PURPOSE, NOR DO THEY ASSUME ANY LIABILITY OR RESPONSIBILITY FOR THE USE OF THIS SOFTWARE.`**

`This software is for research purposes only and has not been approved for clinical use.`

`Software has not been reviewed or approved by the Food and Drug Administration, and is for non-clinical, IRB-approved Research Use Only. In no event shall data or images generated through the use of the Software be used in the provision of patient care.`
  
`YOU MAY NOT DISTRIBUTE COPIES of this software, or copies of software derived from this software, to others outside your organization without specific prior written permission from the CERR development team except where noted for specific software products.`

`All Technology and technical data delivered under this Agreement are subject to US export control laws and may be subject to export or import regulations in other countries. You agree to comply strictly with all such laws and regulations and acknowledge that you have the responsibility to obtain
such licenses to export, re-export, or import as may be required after delivery to you.`

**`You may publish papers and books using results produced using software provided you cite the following`**:
  
  * **AI models**: https://doi.org/10.1088/1361-6560/ac4000  
      https://doi.org/10.48550/arXiv.1909.05054  
                     
  * **CERR model library**: https://doi.org/10.1016/j.ejmp.2020.04.011  



# Downloads

### Install ***pyCERR***

pyCERR handles data import/export, pre- and post-processing transformations for this auto-segmentation model.

In [ ]:
%%capture
!pip install "pyCERR[napari] @ git+https://github.com/cerr/pyCERR.git@testing"


## Prepare planning CTs (DICOM)

In [ ]:
import os
workDir = r'/content' # For Colab

 ### We will use the sample head & neck CT dataset distributed with pyCERR for this demo.

In [ ]:
inputDicomPath = '/usr/local/lib/python3.10/dist-packages/cerr/datasets/sample_ct/head_and_neck'

### Alternatives

 Uncomment the following cell to download data from user-specified ***dataUrl*** to ***dataDownloadDir***.

See also: [demo notebook](github.com/cerr/pyCERR-Notebooks/download_data_from_xnat.ipynb) for downloading data from XNAT.

In [ ]:
# dataUrl = 'http://path.to/data'
# dataDownloadDir = os.path.join(workDir, 'sampleData')
# os.makedirs(dataDownloadDir, exist_ok=True)
# ! wget -O sampleData.gz -L {dataUrl}
# ! tar xf sampleData.gz -C {dataDownloadDir}
# ! rm sampleData.gz

# #Paths to input data
# inputDicomPath = os.path.join(dataDownloadDir,'your_dir_name_here')  # Replace with apropriate path to your dataset

### Define paths to input DICOM directory, desired output directory, and a session directory to store temporary files during model execution

In [ ]:
# Paths to temporary session dierectory for intermediate processing steps
AIoutputPath = os.path.join(workDir, 'AIoutput')
sessionPathDeepLab = os.path.join(workDir, 'tempDL')
sessionPathSelfAttn = os.path.join(workDir, 'tempSA')

if not os.path.exists(AIoutputPath):
  os.mkdir(AIoutputPath)

if not os.path.exists(sessionPathDeepLab):
  os.mkdir(sessionPathDeepLab)
if not os.path.exists(sessionPathSelfAttn):
  os.mkdir(sessionPathSelfAttn)

## Download pre-trained model, inference script, and packaged conda evnironment to ***scriptInstallDir***


In [ ]:
%%capture

# Download model installer
os.chdir(workDir)
!git clone https://github.com/cerr/model_installer.git
os.chdir(os.path.join(workDir,'model_installer'))

# Install CT HN OAR model
!./installer.sh
modelOpt = '6'  # CT_HeadAndNeck_OARs
pythonOpt = 'C' # Download packaged Conda environment

! source ./installer.sh -m {modelOpt} -d {workDir} -p {pythonOpt}

In [ ]:
# Location of inference script
scriptInstallDir = os.path.join(workDir, 'CT_HeadAndNeck_OARs')
deeplabScriptPath = os.path.join(scriptInstallDir,
                         'model_wrapper',
                         'run_inference_deeplab_nii.py')
selfattnScriptPath = os.path.join(scriptInstallDir,
                         'model_wrapper',
                         'run_inference_selfattn_nii.py')


# Location of Conda archive
condaEnvPath = os.path.join(scriptInstallDir, 'conda-pack')

# Location of activation script for Conda environment
activateScript = os.path.join(condaEnvPath,'bin','activate')

### Map structure names to labels

In [ ]:
label_dict1 = {1: "Left masseter", 2: "Right masseter",
               3: "Left medial pterygoid", 4: "Right medial pterygoid"}
label_dict2 = {1: "Larynx"}
label_dict3 = {1: "Pharyngeal constrictor"}
label_dict4 = {1:"Left Parotid", 2:"Right Parotid",
               3:"Left Submandible", 4:"Right Submandible",
               7:"Mandible", 8:"Spinal cord",
               9:"Brain stem", 10:"Oral cavity"}
label_dict_list = [label_dict1, label_dict2, label_dict3, label_dict4]

# Segment OARs

## Apply AI model  to all MR scans

### located in ***inputDicomPath*** and write auto-segmentation results to ***AIoutputPath***

In [ ]:
%%capture
import subprocess
import numpy as np

import cerr.plan_container as pc
from cerr.dataclasses import scan as cerrScn
from cerr.dcm_export import rtstruct_iod
from cerr.utils.ai_pipeline import createSessionDir, getScanNumFromIdentifier


modality = 'CT'
scanNum = 0

# Loop over pyCERR files
fileList = os.listdir(inputDicomPath)
numFiles = len(fileList)

for iFile in range(numFiles):

    inputFilename = fileList[iFile]
    dcmDir = os.path.join(inputDicomPath, inputFilename)

    # Identify CT scan
    identifier = {'imageType':'CT SCAN'}
    planC = pc.loadDcmDir(dcmDir)
    scanNum = getScanNumFromIdentifier(identifier, planC)[0]

    #---------------------
    # Run DeepLab models
    #----------------------
    # Create session dir to store temporary data
    modInputPath, __ = createSessionDir(sessionPathDeepLab,
                                                   inputDicomPath)
    # Export scan to NII
    scanFilename = os.path.join(modInputPath,
                                f"{inputFilename}_scan_3D.nii.gz")
    planC.scan[scanNum].saveNii(scanFilename)
    # AI inference
    result = subprocess.run(f"source {activateScript} && python {deeplabScriptPath}\
                  {modInputPath} {sessionPathDeepLab} {AIoutputPath}",\
                   capture_output=False, shell=True, executable="/bin/bash")
    #-------------------------
    # Run self-attention model
    #-------------------------
    # Create session dir to store temporary data
    modInputPath, __ = createSessionDir(sessionPathSelfAttn,
                                        inputDicomPath)
    # Export scan to NII
    scanFilename = os.path.join(modInputPath,
                                f"{inputFilename}_scan_3D.nii.gz")
    planC.scan[scanNum].saveNii(scanFilename)

    # AI inference
    result = subprocess.run(f"source {activateScript} && python {selfattnScriptPath}\
                  {modInputPath} {sessionPathSelfAttn} {AIoutputPath}",\
                   capture_output=False, shell=True, executable="/bin/bash")

## **Optional**: Uncomment the following to download the output segmentations to your workspace bucket.

In [ ]:
# workspaceBucket = os.environ['WORKSPACE_BUCKET']
# !gcloud storage cp -r {outputDicomPath} {workspaceBucket}

# Display results

## Overlay AI segmentations on scan for visualization using ***Matplotlib***

Note: This example displays the last segmented dataset by default.    
Load the appropriate pyCERR archive to `planC` to view results for desired dataset.

In [ ]:
%%capture
# Import AI-genereated masks
maskFiles = ['chew_AI_seg','larynx_AI_seg','cm_AI_seg','self_attn_AI_seg']
origNumStructs = len(planC.structure)

for maskNum in range(len(maskFiles)):
  maskFileName = f"{inputFilename}_scan_3D_{maskFiles[maskNum]}.nii.gz"
  maskFilePath = os.path.join(AIoutputPath,maskFileName)
  planC = pc.loadNiiStructure(maskFilePath, scanNum, planC, labels_dict=label_dict_list[maskNum])

newNumStructs = len(planC.structure)

In [ ]:
from cerr.viewer import showMplNb

showMplNb(planC=planC, scan_nums=scanNum,
          struct_nums=np.arange(origNumStructs,newNumStructs),
          windowCenter=50, windowWidth=450)

Output()